# Segmenting and Clustering Neighborhoods in Toronto
## Part A

In [11]:
import requests
import pandas as pd
import numpy as np
import seaborn as sns
pd.set_option('display.max_columns', None)

<font color='Blue'>Assign the link of the website to a variable named 'url'</font>

In [12]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

<font color='Blue'>Read the source code and create a BeautifulSoup object </font>

In [61]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(url,'lxml')
#print(soup.prettify())

<font color='Blue'>All the table contents which we intend to extract is under class Wikitable Sortable </font>

<font color='green'>Then extract all the rows within 'tr' </font>

In [63]:
My_table = soup.find('table',{'class':'wikitable sortable'})
rows=My_table.findAll('tr')
#rows

<font color='Blue'>Define an empty data frame and append the information within 'td' row by row </font>

In [16]:
Mydata = []
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    Mydata.append([ele for ele in cols if ele])

In [17]:
pd.DataFrame(Mydata).head()

,0,1,2
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


<font color='Blue'> Rename columns and drop the first row </font>

In [18]:
WikiData=pd.DataFrame(Mydata, columns=['PostalCode', 'Borough', 'Neighborhood'])
WikiData.drop(0, inplace=True)
print(WikiData.shape)
WikiData.head()

(287, 3)


,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


<font color='Blue'>Ignore cells with a borough that is Not assigned</font>

<font color='green'>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough</font>

In [19]:
WikiData.drop(WikiData[WikiData['Borough'] == 'Not assigned'].index, inplace = True) 
# 
WikiData['Neighborhood'].loc[(WikiData['Borough'] != 'Not assigned') & (WikiData['Neighborhood'] == 'Not assigned')]=WikiData['Borough']
print(WikiData.shape)
WikiData.head()

(210, 3)


,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


<font color='Blue'>Make the wide format of the data, ignoring 'Borough' for now and Replace the 'NaN' values with " "</font>

In [26]:
# 
WikiData2=WikiData.pivot(index='PostalCode', columns='Neighborhood', values='Neighborhood')
# 
WikiData2.fillna('', inplace=True)
WikiData2.reset_index(inplace=True)
WikiData2.tail()

Neighborhood,PostalCode,Adelaide,Agincourt,Agincourt North,Albion Gardens,Alderwood,Bathurst Manor,Bathurst Quay,Bayview Village,Beaumond Heights,Bedford Park,Berczy Park,Birch Cliff,Bloordale Gardens,Brockton,Business Reply Mail Processing Centre 969 Eastern,CFB Toronto,CN Tower,Cabbagetown,Caledonia-Fairbanks,Canada Post Gateway Processing Centre,Cedarbrae,Central Bay Street,Chinatown,Christie,Church and Wellesley,Clairlea,Clarks Corners,Cliffcrest,Cliffside,Cliffside West,Cloverdale,Commerce Court,Davisville,Davisville North,Deer Park,Del Ray,Design Exchange,Don Mills North,Don Mills South,Dorset Park,Dovercourt Village,Downsview,Downsview Central,Downsview East,Downsview North,Downsview Northwest,Downsview West,Dufferin,East Birchmount Park,East Toronto,Emery,Eringate,Exhibition Place,Fairview,First Canadian Place,Flemingdon Park,Forest Hill North,Forest Hill SE,Forest Hill West,Garden District,Glencairn,Golden Mile,Grange Park,Guildwood,Harbord,Harbourfront,Harbourfront East,Harbourfront West,Henry Farm,High Park,Highland Creek,Hillcrest Village,Humber Bay,Humber Bay Shores,Humber Summit,Humbergate,Humberlea,Humewood-Cedarvale,India Bazaar,Ionview,Island airport,Islington,Jamestown,Keelesdale,Kennedy Park,Kensington Market,King,King and Spadina,King's Mill Park,Kingsview Village,Kingsway Park South East,Kingsway Park South West,L'Amoreaux East,L'Amoreaux West,Lawrence Heights,Lawrence Manor,Lawrence Manor East,Lawrence Park,Leaside,Little Portugal,Long Branch,Malvern,Markland Wood,Martin Grove,Martin Grove Gardens,Maryvale,Milliken,Mimico NE,Mimico NW,Mimico South,Montgomery Road,Moore Park,Morningside,Mount Dennis,Mount Olive,New Toronto,Newtonbrook,North Midtown,North Park,North Toronto West,Northwest,Northwood Park,Oakridge,Old Burnhamthorpe,Old Mill North,Old Mill South,Oriole,Parkdale,Parkdale Village,Parkview Hill,Parkwoods,Port Union,Princess Gardens,Queen's Park,Railway Lands,Rathnelly,Richmond,Richview Gardens,Riverdale,Roncesvalles,Rosedale,Roselawn,Rouge,Rouge Hill,Royal York South East,Royal York South West,Runnymede,Ryerson,Scarborough Town Centre,Scarborough Village,Scarborough Village West,Silver Hills,Silverstone,Silverthorn,South Hill,South Niagara,South Steeles,South of Bloor,St. James Town,St. Phillips,Steeles East,Stn A PO Boxes 25 The Esplanade,Studio District,Sullivan,Summerhill East,Summerhill West,Sunnylea,Swansea,Tam O'Shanter,The Annex,The Beaches,The Beaches West,The Danforth West,The Junction North,The Junction South,The Kingsway,The Queensway East,The Queensway West,Thistletown,Thorncliffe Park,Toronto Dominion Centre,Toronto Islands,Trinity,Underground city,Union Station,University of Toronto,Upper Rouge,Upwood Park,Victoria Hotel,Victoria Village,West Deane Park,West Hill,Westmount,Weston,Wexford,Wexford Heights,Willowdale,Willowdale South,Willowdale West,Wilson Heights,Woburn,Woodbine Gardens,Woodbine Heights,York Mills,York Mills West,York University,Yorkville
98,M9N,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Weston,,,,,,,,,,,,,
99,M9P,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Westmount,,,,,,,,,,,,,,
100,M9R,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Kingsview Village,,,,,,,,,,,,,,,Martin Grove Gardens,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Richview Gardens,,,,,,,,,,,,,,,,,,,,,,St. Phillips,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101,M9V,,,,Albion Gardens,,,,,Beaumond Heights,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Humbergate,,,,,,,Jamestown,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Mount Olive,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Silverstone,,,,South Steeles,,,,,,,,,,,,,,,,,,,,,,Thistletown,,,,,,,,,,,,,,,,,,,,,,,,,,,,
102,M9W,,,,,,,,,

<font color='Blue'>Concat columns row by row with a ',' in between</font>
<font color='Blue'>remove the last ', ' in the column</font>

In [27]:
# 
WikiData2['Neighborhoods']=''
for row in range(0,103):
    for index in range(1,208):
        if WikiData2.iloc[row,index] != '':
            temp=WikiData2.iloc[row,index]
            WikiData2['Neighborhoods'][row]= WikiData2['Neighborhoods'][row]+temp+', '
WikiData3=WikiData2[['PostalCode','Neighborhoods']].copy()
WikiData3['Neighborhoods'] = WikiData3['Neighborhoods'].map(lambda x: str(x)[:-2])
# Add the Borough column
WikiData3.head()

Neighborhood,PostalCode,Neighborhoods
0,M1B,"Malvern, Rouge"
1,M1C,"Highland Creek, Port Union, Rouge Hill"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


<font color='Blue'>Merge with original Data to add Borough and then remove duplicates </font>

In [57]:
WikiData4=pd.merge(WikiData3, WikiData[['PostalCode','Borough']], left_on=['PostalCode'],
                   right_on=['PostalCode'],  how='inner')
WikiData4.drop_duplicates(inplace=True)
WikiData4.reset_index(drop=True, inplace=True)
WikiData4.head()

,PostalCode,Neighborhoods,Borough
0,M1B,"Malvern, Rouge",Scarborough
1,M1C,"Highland Creek, Port Union, Rouge Hill",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough


<font color='Blue'> Clean the Dataframe</font>

In [58]:
#
WikiData4.insert(3, 'Neighborhood', WikiData4['Neighborhoods'].values)
WikiData4.drop(columns=['Neighborhoods'], inplace=True)
pd.set_option('display.max_rows', None)
WikiData4.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [426]:
WikiData4.shape

(103, 3)